In [1]:
import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                #rescale = 1/255.0,  # Rescale... 0 to 1 .. 0 is white colors and 1 is black colors
                                                                rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.20,
                                                                horizontal_flip=True,
                                                                vertical_flip=True,
                                                                validation_split=0.15, # dont need a validate data generator
                                                                preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
                                                                )



test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                #rescale = 1/255.0,  # Rescale... 0 to 1 .. 0 is white colors and 1 is black colors
                                                                rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.20,
                                                                horizontal_flip=True,
                                                                vertical_flip=True,
                                                                validation_split=0.15,
                                                                preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
                                                                )


train_generator = train_datagen.flow_from_directory(
                                                    directory='seg_test',
                                                    target_size=(224, 224),
                                                    batch_size=12,
                                                    class_mode='categorical',
                                                    subset='training', # we split the trian data set. thats why we need this
                                                    shuffle=True,
                                                    seed=42
                                                    )

test_generator = test_datagen.flow_from_directory(
                                                    directory='seg_train',
                                                    target_size=(224, 224),
                                                    batch_size=12,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    seed=42
                                                    )

valid_generator = train_datagen.flow_from_directory (
    
                                                    directory='seg_train',
                                                    target_size=(224, 224),
                                                    batch_size=12,
                                                    class_mode='categorical',
                                                    subset='validation',
                                                    shuffle=True,
                                                    seed=42
                                                                    
                                                            
    
)


Found 2553 images belonging to 6 classes.
Found 14034 images belonging to 6 classes.
Found 2102 images belonging to 6 classes.


In [7]:
pretrained_model = tf.keras.applications.InceptionV3(
                                                        include_top=False, # should be false when we use custom inputshape
                                                        weights="imagenet",
                                                        input_shape=(224,224,3)
                                                    )


87910968/87910968 [==============================] - 79s 1us/step


In [10]:
pretrained_model.trainable = False
inputs = tf.keras.Input(shape=(224, 224, 3))
x = pretrained_model(inputs, training=True)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

outputs = tf.keras.layers.Dense(6, activation='softmax')(x) # output classes

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_3 (Dropout)         (None, 2048)              0         
                                                                 
 dense_3 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                           

In [11]:
model.compile(
            loss = 'categorical_crossentropy',
            optimizer = 'adam',
            metrics=['accuracy']
            )

model.fit(
        train_generator,
        validation_data= valid_generator,
        epochs=10
        )

Epoch 1/10
213/213 [==============================] - 146s 672ms/step - loss: 1.4544 - accuracy: 0.4685 - val_loss: 0.7061 - val_accuracy: 0.7179
Epoch 2/10
213/213 [==============================] - 91s 429ms/step - loss: 0.8596 - accuracy: 0.6608 - val_loss: 0.5526 - val_accuracy: 0.8030
Epoch 3/10
213/213 [==============================] - 91s 429ms/step - loss: 0.7549 - accuracy: 0.7070 - val_loss: 0.5319 - val_accuracy: 0.7978
Epoch 4/10
213/213 [==============================] - 90s 423ms/step - loss: 0.7468 - accuracy: 0.7121 - val_loss: 0.5431 - val_accuracy: 0.8030
Epoch 5/10
213/213 [==============================] - 88s 411ms/step - loss: 0.6949 - accuracy: 0.7278 - val_loss: 0.5157 - val_accuracy: 0.8187
Epoch 6/10
213/213 [==============================] - 84s 394ms/step - loss: 0.6368 - accuracy: 0.7583 - val_loss: 0.5452 - val_accuracy: 0.7945
Epoch 7/10
213/213 [==============================] - 84s 393ms/step - loss: 0.6326 - accuracy: 0.7599 - val_loss: 0.4947 - val_a

In [12]:
class_dict = train_generator.class_indices
class_dict_rev = {v : k for k, v in class_dict.items()}

In [16]:
def inference(image_path, model):
    img = cv.imread(image_path)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (224, 224))
    img = img / 255.0
    img = np.array([img])
    P = model.predict(img).squeeze()
    class_ = P.argmax()
    return class_dict_rev[class_], P.max()


inference('seg_pred/seg_pred/3.jpg', model)

1/1 [==============================] - 0s 51ms/step


('street', 0.96313787)